**Programador:** Marco Gutierrez. Si se tiene alguna consulta, mandarla al siguiente [email](mailto:mgutierrezc@up.edu.pe)

# Obteniendo la información para el Scrapping

Importamos las librerias para el scrapping

In [30]:
import pandas as pd
import string
import googlemaps
import numpy as np
import os
import re

## Importando las provincias/distritos a analizar

Primero, leeremos un archivo que contiene los elementos de los ubigeos del INEI. Se puede descargar entrando [aquí](http://webinei.inei.gob.pe:8080/sisconcode/proyecto/index.htm?proyectoTitulo=UBIGEO&proyectoId=3) y dando click a la opción **Excel** en la pestaña *Busqueda por Ubicación Geográfica*

In [31]:
ubigeos = pd.read_excel('ubigeo_inei.xls',skiprows=1)
ubigeos.head()

,Unnamed: 0,DEPARTAMENTO,Unnamed: 2,Unnamed: 3,PROVINCIA,DISTRITO
0,NaN,01 Amazonas,NaN,NaN,,
1,NaN,01 Amazonas,NaN,NaN,01 Chachapoyas,
2,NaN,01 Amazonas,NaN,NaN,01 Chachapoyas,01 Chachapoyas
3,NaN,01 Amazonas,NaN,NaN,01 Chachapoyas,02 Asunción
4,NaN,01 Amazonas,NaN,NaN,01 Chachapoyas,03 Balsas


Necesitamos las provincias y distritos de esta base, así que limpiaremos los valores de estas variables

In [32]:
ubigeos["DISTRITO"] = ubigeos["DISTRITO"].str.replace('^\d* ', "", flags=re.I, regex=True)
ubigeos["PROVINCIA"] = ubigeos["PROVINCIA"].str.replace('^\d* ', "", flags=re.I, regex=True)
ubigeos["DEPARTAMENTO"] = ubigeos["DEPARTAMENTO"].str.replace('^\d* ', "", flags=re.I, regex=True)
ubigeos.head()

,Unnamed: 0,DEPARTAMENTO,Unnamed: 2,Unnamed: 3,PROVINCIA,DISTRITO
0,NaN,Amazonas,NaN,NaN,,
1,NaN,Amazonas,NaN,NaN,Chachapoyas,
2,NaN,Amazonas,NaN,NaN,Chachapoyas,Chachapoyas
3,NaN,Amazonas,NaN,NaN,Chachapoyas,Asunción
4,NaN,Amazonas,NaN,NaN,Chachapoyas,Balsas


Limpiando las listas

Necesitaremos estas dos columnas como listas, pues después las usaremos para crear un DataFrame donde guardaremos los resultados del scrapping

In [33]:
# Creando las listas
provincias = list(ubigeos['PROVINCIA'])
distritos = list(ubigeos['DISTRITO'] + ", " + ubigeos['PROVINCIA'] + ", " + ubigeos['DEPARTAMENTO'])
for_removal_dist = list(", " + ubigeos['PROVINCIA'] + ", " + ubigeos['DEPARTAMENTO'])

distritos

[', , Amazonas',
 ', Chachapoyas, Amazonas',
 'Chachapoyas, Chachapoyas, Amazonas',
 'Asunción, Chachapoyas, Amazonas',
 'Balsas, Chachapoyas, Amazonas',
 'Cheto, Chachapoyas, Amazonas',
 'Chiliquin, Chachapoyas, Amazonas',
 'Chuquibamba, Chachapoyas, Amazonas',
 'Granada, Chachapoyas, Amazonas',
 'Huancas, Chachapoyas, Amazonas',
 'La Jalca, Chachapoyas, Amazonas',
 'Leimebamba, Chachapoyas, Amazonas',
 'Levanto, Chachapoyas, Amazonas',
 'Magdalena, Chachapoyas, Amazonas',
 'Mariscal Castilla, Chachapoyas, Amazonas',
 'Molinopampa, Chachapoyas, Amazonas',
 'Montevideo, Chachapoyas, Amazonas',
 'Olleros, Chachapoyas, Amazonas',
 'Quinjalca, Chachapoyas, Amazonas',
 'San Francisco de Daguas, Chachapoyas, Amazonas',
 'San Isidro de Maino, Chachapoyas, Amazonas',
 'Soloco, Chachapoyas, Amazonas',
 'Sonche, Chachapoyas, Amazonas',
 ', Bagua, Amazonas',
 'Bagua, Bagua, Amazonas',
 'Aramango, Bagua, Amazonas',
 'Copallin, Bagua, Amazonas',
 'El Parco, Bagua, Amazonas',
 'Imaza, Bagua, Amazon

In [34]:
# Eliminando duplicados
provincias = set(provincias)
distritos = set(distritos)
for_removal_dist = set(for_removal_dist)

# Eliminando algunos elementos inutiles ('', nan)
provincias = list(filter(lambda x: str(x) != 'nan', provincias))
distritos = list(filter(lambda x: str(x) != 'nan', distritos))

provincias.remove('')
provincias.remove('PROVINCIA')
distritos = [distr for distr in distritos if distr not in for_removal_dist]
distritos.remove('DISTRITO, PROVINCIA, DEPARTAMENTO')
distritos


['Coya, Calca, Cusco',
 'San Juan de la Virgen, Tumbes , Tumbes',
 'Alto Laran, Chincha , Ica',
 'Andoas, Datem del Marañón , Loreto',
 'Castrovirreyna, Castrovirreyna, Huancavelica',
 'Aurahua, Castrovirreyna, Huancavelica',
 'Cabanillas, San Román , Puno',
 'San Juan Bautista, Ica , Ica',
 'Eten, Chiclayo , Lambayeque',
 'San Borja, Lima , Lima',
 'Churcampa, Churcampa, Huancavelica',
 'Paramonga, Barranca , Lima',
 'Alcamenca, Víctor Fajardo, Ayacucho',
 'Huancapi, Víctor Fajardo, Ayacucho',
 'Soloco, Chachapoyas, Amazonas',
 'San Mateo, Huarochirí , Lima',
 'Totos, Cangallo, Ayacucho',
 'Vizcatan del Ene, Satipo , Junín',
 'Santa María del Mar, Lima , Lima',
 'Santa, Santa, Áncash',
 'Sucre, Celendín, Cajamarca',
 'Yuracmarca, Huaylas, Áncash',
 'Cajaruro, Utcubamba, Amazonas',
 'Huacho, Huaura , Lima',
 'Morococha, Yauli , Junín',
 'Guadalupito, Virú , La Libertad',
 'Pacora, Lambayeque , Lambayeque',
 'Chisquilla, Bongará, Amazonas',
 'Utco, Celendín, Cajamarca',
 'Colquepata, Pa

Eliminando las tildes

In [35]:
# Definimos una función para eliminar las tildes
def normalize(s):
    replacements = (
        ("á", "a"),
        ("à", "a"),
        ("é", "e"),
        ("è", "e"),
        ("í", "i"),
        ("ì", "i"),
        ("ó", "o"),
        ("ò", "o"),
        ("ú", "u"),
        ("ù", "u")
    )
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    return s

In [36]:
# La aplicamos a cada elemento de nuestras provincias/distritos
nro_provs = len(provincias)
for i in range(nro_provs):
    provincias[i] = normalize(provincias[i])
    # De paso, les quitaremos espacios
    provincias[i] = provincias[i].strip()

nro_distrs = len(distritos)
for i in range(nro_distrs):
    distritos[i] = normalize(distritos[i])
    distritos[i] = distritos[i].split(",")
    
    # Eliminando los espacios dentro de los elementos de un distrito único (distr, prov, dept)
    item_distr = len(distritos[i])
    empty_distr = []
    for j in range(item_distr):
        distritos[i][j] = distritos[i][j].strip()
        empty_distr.append(distritos[i][j])
        
    # Joining our separated districts
    distritos[i] = '+'.join(empty_distr)
    print(distritos[i])

Coya+Calca+Cusco
San Juan de la Virgen+Tumbes+Tumbes
Alto Laran+Chincha+Ica
Andoas+Datem del Marañon+Loreto
Castrovirreyna+Castrovirreyna+Huancavelica
Aurahua+Castrovirreyna+Huancavelica
Cabanillas+San Roman+Puno
San Juan Bautista+Ica+Ica
Eten+Chiclayo+Lambayeque
San Borja+Lima+Lima
Churcampa+Churcampa+Huancavelica
Paramonga+Barranca+Lima
Alcamenca+Victor Fajardo+Ayacucho
Huancapi+Victor Fajardo+Ayacucho
Soloco+Chachapoyas+Amazonas
San Mateo+Huarochiri+Lima
Totos+Cangallo+Ayacucho
Vizcatan del Ene+Satipo+Junin
Santa Maria del Mar+Lima+Lima
Santa+Santa+Ancash
Sucre+Celendin+Cajamarca
Yuracmarca+Huaylas+Ancash
Cajaruro+Utcubamba+Amazonas
Huacho+Huaura+Lima
Morococha+Yauli+Junin
Guadalupito+Viru+La Libertad
Pacora+Lambayeque+Lambayeque
Chisquilla+Bongara+Amazonas
Utco+Celendin+Cajamarca
Colquepata+Paucartambo+Cusco
San Juan de Lopecancha+Luya+Amazonas
San Pedro de Huancayre+Huarochiri+Lima
Rosa Panduro+Putumayo+Loreto
Santa Catalina de Mossa+Morropon+Piura
Chilca+Cañete+Lima
Constitucion+

In [37]:
# print(provincias)
print(distritos)

['Coya+Calca+Cusco', 'San Juan de la Virgen+Tumbes+Tumbes', 'Alto Laran+Chincha+Ica', 'Andoas+Datem del Marañon+Loreto', 'Castrovirreyna+Castrovirreyna+Huancavelica', 'Aurahua+Castrovirreyna+Huancavelica', 'Cabanillas+San Roman+Puno', 'San Juan Bautista+Ica+Ica', 'Eten+Chiclayo+Lambayeque', 'San Borja+Lima+Lima', 'Churcampa+Churcampa+Huancavelica', 'Paramonga+Barranca+Lima', 'Alcamenca+Victor Fajardo+Ayacucho', 'Huancapi+Victor Fajardo+Ayacucho', 'Soloco+Chachapoyas+Amazonas', 'San Mateo+Huarochiri+Lima', 'Totos+Cangallo+Ayacucho', 'Vizcatan del Ene+Satipo+Junin', 'Santa Maria del Mar+Lima+Lima', 'Santa+Santa+Ancash', 'Sucre+Celendin+Cajamarca', 'Yuracmarca+Huaylas+Ancash', 'Cajaruro+Utcubamba+Amazonas', 'Huacho+Huaura+Lima', 'Morococha+Yauli+Junin', 'Guadalupito+Viru+La Libertad', 'Pacora+Lambayeque+Lambayeque', 'Chisquilla+Bongara+Amazonas', 'Utco+Celendin+Cajamarca', 'Colquepata+Paucartambo+Cusco', 'San Juan de Lopecancha+Luya+Amazonas', 'San Pedro de Huancayre+Huarochiri+Lima', 'Ro

# Scrapping de Distancias y Tiempos de viaje

Para realizar este scrapping, se tomará como referencia esta [guía](https://www.datahubbs.com/google-maps-python/)

## Matrices base a nivel de Provincias

Lo primero será crear un DataFrame con los distritos/provincias de origen y de llegada.

In [38]:
first_col = ["Origin Province\Destination Province"]
prov_distances = pd.DataFrame({"Origin Province\Destination Province": provincias}, 
                        columns = first_col+provincias)
prov_times = pd.DataFrame({"Origin Province\Destination Province": provincias}, 
                        columns = first_col+provincias)
prov_distances

,Origin Province\Destination Province,Marañon,Pisco,Chachapoyas,Pasco,Rodriguez de Mendoza,Aymaraes,Aija,Yungay,Manu,...,Lambayeque,Luya,Castrovirreyna,Ferreñafe,Tumbes,Datem del Marañon,Pomabamba,San Pablo,Ambo,Acomayo
0,Marañon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pisco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chachapoyas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Pasco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Rodriguez de Mendoza,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,Datem del Marañon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192,Pomabamba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193,San Pablo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194,Ambo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Creando listas con los origenes/destinos para reemplazar en el dataframe

In [39]:
# Como queremos iterar sobre los nombres de las provincias
origen_destino_prov=[]

for prov_row in provincias:
    origen_destino_prov.append(prov_row+"+"+"Peru")

origen_destino_prov

['Marañon+Peru',
 'Pisco+Peru',
 'Chachapoyas+Peru',
 'Pasco+Peru',
 'Rodriguez de Mendoza+Peru',
 'Aymaraes+Peru',
 'Aija+Peru',
 'Yungay+Peru',
 'Manu+Peru',
 'Talara+Peru',
 'Huancayo+Peru',
 'Huacaybamba+Peru',
 'Chepen+Peru',
 'Paucar del Sara Sara+Peru',
 'Espinar+Peru',
 'Calca+Peru',
 'Bolivar+Peru',
 'El Collao+Peru',
 'General Sanchez Cerro+Peru',
 'Pachitea+Peru',
 'Huamanga+Peru',
 'Huanuco+Peru',
 'Huamalies+Peru',
 'Contumaza+Peru',
 'Jorge Basadre+Peru',
 'Chiclayo+Peru',
 'Lauricocha+Peru',
 'Carhuaz+Peru',
 'Santa Cruz+Peru',
 'Chincha+Peru',
 'Viru+Peru',
 'Canas+Peru',
 'Tocache+Peru',
 'Sullana+Peru',
 'Leoncio Prado+Peru',
 'Pallasca+Peru',
 'Mariscal Caceres+Peru',
 'Caylloma+Peru',
 'Tayacaja+Peru',
 'Chincheros+Peru',
 'Grau+Peru',
 'Hualgayoc+Peru',
 'Canchis+Peru',
 'Quispicanchi+Peru',
 'Santa+Peru',
 'Nasca+Peru',
 'Huarochiri+Peru',
 'Sechura+Peru',
 'Melgar+Peru',
 'Purus+Peru',
 'Picota+Peru',
 'Cangallo+Peru',
 'Oxapampa+Peru',
 'Bongara+Peru',
 'Huari+P

## Matrices base a nivel de distritos

Crearemos nuestra matriz de la siguiente manera

In [40]:
first_col = ["Origin Province\Destination Province"]
dist_distances = pd.DataFrame({"Origin Province\Destination Province": distritos}, 
                        columns = first_col+distritos)
dist_times = pd.DataFrame({"Origin Province\Destination Province": distritos}, 
                        columns = first_col+distritos)
dist_distances

,Origin Province\Destination Province,Coya+Calca+Cusco,San Juan de la Virgen+Tumbes+Tumbes,Alto Laran+Chincha+Ica,Andoas+Datem del Marañon+Loreto,Castrovirreyna+Castrovirreyna+Huancavelica,Aurahua+Castrovirreyna+Huancavelica,Cabanillas+San Roman+Puno,San Juan Bautista+Ica+Ica,Eten+Chiclayo+Lambayeque,...,San Pedro de Coris+Churcampa+Huancavelica,Santa Rosa+Chiclayo+Lambayeque,Huambo+Caylloma+Arequipa,Ocumal+Luya+Amazonas,Mochumi+Lambayeque+Lambayeque,Candarave+Candarave+Tacna,Campoverde+Coronel Portillo+Ucayali,Jorge Chavez+Celendin+Cajamarca,Progreso+Grau+Apurimac,Sayapullo+Gran Chimu+La Libertad
0,Coya+Calca+Cusco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,San Juan de la Virgen+Tumbes+Tumbes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alto Laran+Chincha+Ica,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Andoas+Datem del Marañon+Loreto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Castrovirreyna+Castrovirreyna+Huancavelica,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1869,Candarave+Candarave+Tacna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1870,Campoverde+Coronel Portillo+Ucayali,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1871,Jorge Chavez+Celendin+Cajamarca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1872,Progreso+Grau+Apurimac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Creando listas con los origenes/destinos para reemplazar en el dataframe

In [41]:
# Como queremos iterar sobre los nombres de las provincias
origen_destino_dist=[]

for dist_row in distritos:
    origen_destino_dist.append(dist_row+"+"+"Peru")

origen_destino_dist

['Coya+Calca+Cusco+Peru',
 'San Juan de la Virgen+Tumbes+Tumbes+Peru',
 'Alto Laran+Chincha+Ica+Peru',
 'Andoas+Datem del Marañon+Loreto+Peru',
 'Castrovirreyna+Castrovirreyna+Huancavelica+Peru',
 'Aurahua+Castrovirreyna+Huancavelica+Peru',
 'Cabanillas+San Roman+Puno+Peru',
 'San Juan Bautista+Ica+Ica+Peru',
 'Eten+Chiclayo+Lambayeque+Peru',
 'San Borja+Lima+Lima+Peru',
 'Churcampa+Churcampa+Huancavelica+Peru',
 'Paramonga+Barranca+Lima+Peru',
 'Alcamenca+Victor Fajardo+Ayacucho+Peru',
 'Huancapi+Victor Fajardo+Ayacucho+Peru',
 'Soloco+Chachapoyas+Amazonas+Peru',
 'San Mateo+Huarochiri+Lima+Peru',
 'Totos+Cangallo+Ayacucho+Peru',
 'Vizcatan del Ene+Satipo+Junin+Peru',
 'Santa Maria del Mar+Lima+Lima+Peru',
 'Santa+Santa+Ancash+Peru',
 'Sucre+Celendin+Cajamarca+Peru',
 'Yuracmarca+Huaylas+Ancash+Peru',
 'Cajaruro+Utcubamba+Amazonas+Peru',
 'Huacho+Huaura+Lima+Peru',
 'Morococha+Yauli+Junin+Peru',
 'Guadalupito+Viru+La Libertad+Peru',
 'Pacora+Lambayeque+Lambayeque+Peru',
 'Chisquilla+B

## ¿Cómo usar el paquete `googlemaps`?

Ahora que ya tenemos todos los datos listos, nos falta activar la clase `googlemaps` para que podamos extraer tiempos y distancias. Ello se puede hacer así:

1. Importamos la librería googlemaps (Hecho al inicio de este jupyter)
1. Activamos el API `Distance Matrix API` en Google Cloud Platform (se puede encontrar una guía [aquí](https://www.youtube.com/watch?v=n1UorU1PALk))
1. Creamos una API key para poder usar el item anterior (guía [aquí](https://www.youtube.com/watch?v=2_HZObVbe-g))
1. Activaremos el cobro por el uso del API (Nos daran una prueba gratuita. La guía está [aquí](https://support.google.com/googleapi/answer/6158867?hl=en))

Sin un API key no podrán seguir el proceso, pero es de uso **personal**. Por ello, nunca la debemos escribir directamente en nuestro código; sin embargo, aprenderemos como utilizarla manteniendola privada y lo haremos usando **variables del sistema**

En el Menú de Inicio de Windows, busquen "environment" o "variables". Escojan la opción **"Editar las variables de entorno del sistema"**
![](images\env1.jpg)

Abranlo y entren a la pestaña `Opciones Avanzadas`

![](images\env2.jpg)

Hagan click a la opción `Variables de entornno`
![](images\env3.jpg)

Vayan a la sección `Variables de entorno` y denle click a **Nueva...**
![](images\env4.jpg)

Coloquen el nombre que prefieran (en mi caso, usé  `maps_key`) y, como valor, la API Key correspondiente (no la deben colocar entre comillas)
![](images\env5.jpg)

Denle Aceptar a esta ventana y luego a las anteriores hasta que se cierren todas. Cuando estén cerradas, reinicien su computadora y ya deberían poder usarla

## Usando `googlemaps` para crear una función que calcule distancias/tiempos

Cuando ya tengan su API Key lista, llamenla de la siguiente manera

In [42]:
key = os.environ.get('maps_key')

Ahora, activaremos la clase google maps usando nuestro key

In [43]:
gmaps = googlemaps.Client(key)

In [44]:
def matrix_blocks(first_row, first_column, final_row_plus_one, final_col, df_distances, df_times, origin_dest):
    """
    Función para completar la matriz por bloques que comiencen desde "first_row" y "first_column" hasta "iterations_row" 
    y "iterations_column". Se creó porque ejecutar el sgte codigo para todas las celdas toma mucho tiempo en ejecutarse.
    
    Para hacer más eficiente la tarea, se puede ejecutar esto por partes hasta completarlo. Además, permite convertir la tarea
    en una grupal. Un equipo puede hacer que cada uno de sus miembros complete ciertos bloques (e.g. linea 1 hasta 12, otro 
    linea 12 hasta 24, y así sucesivamente)
    """
    # Codigo base para cada fila (comenzando con la 0)
    for row in range(first_row, final_row_plus_one):
        print(row)
        for col in range(first_column, final_col):
            # Definiendo un índice para nuestra lista con los origenes/destinos en función a la columna del dataframe
            # en la que queremos el resultado
            index_destination = col-1

            # Si las filas y columnas son consecutivas, las distancias y tiempo son de un sitio al mismo sitio, asi que 
            # debemos controlar ese factir
            if col-1==row:
                df_distances.iloc[row,col] = '0 km'
                df_times.iloc[row,col] = '0 mins'

            else:
                # Obteniendo las distancias/tiempos correspondientes: 
                dist_time_mat = gmaps.distance_matrix(origins=origin_dest[row], destinations=origin_dest[index_destination], mode='driving')
                distance_duration = dist_time_mat['rows'][0]['elements'][0]

                # No siempre es posible viajar de un lugar a otro por restricciones geográficas, así que debemos definir qué ocurre
                # en casos así
                if distance_duration.get('distance'):
                    df_distances.iloc[row,col] = distance_duration['distance']['text']
                else:
                    df_distances.iloc[row,col] = '-'

                if distance_duration.get('duration'):
                    df_times.iloc[row,col] = distance_duration['duration']['text']
                else:
                    df_times.iloc[row,col] = '-'            

                # Para debuggear:
                # print('Origen: '+origen_destino[row]+' | Destino: '+origen_destino[index_destination]) 
                # print(prov_distances.iloc[row,col])

### Aplicando la función a las Matrices para Provincias

In [29]:
primera_fila = 71
primera_col = 1
ult_fila_mas_uno = len(provincias)
ult_col = len(provincias)+1

# matrix_blocks(primera_fila, primera_col, ult_fila_mas_uno, ult_col, prov_distances, prov_times, origen_destino_prov)

In [149]:
prov_distances

,Origin Province\Destination Province,Chepen,Huarochiri,Espinar,Castrovirreyna,Moyobamba,Caraveli,Palpa,Ocros,Manu,...,Caylloma,Tocache,Puno,La Mar,Huamalies,Putumayo,Urubamba,Sullana,Quispicanchi,Otuzco
0,Chepen,0 km,894 km,"1,776 km","1,196 km",673 km,"1,474 km","1,089 km",592 km,-,...,"1,698 km",673 km,"1,985 km",707 km,634 km,-,"1,801 km",325 km,"1,840 km",218 km
1,Huarochiri,896 km,0 km,"1,096 km",516 km,"1,294 km",794 km,409 km,487 km,-,...,"1,018 km",869 km,"1,305 km",189 km,674 km,-,"1,121 km","1,219 km","1,160 km",828 km
2,Espinar,"1,992 km","1,309 km",0 km,874 km,"2,068 km",663 km,895 km,"1,583 km",-,...,78.5 km,"1,643 km",252 km,"1,284 km","1,769 km",-,284 km,"2,315 km",198 km,"1,924 km"
3,Castrovirreyna,"1,083 km",400 km,"1,007 km",0 km,"1,336 km",705 km,320 km,674 km,-,...,929 km,911 km,"1,216 km",376 km,860 km,-,844 km,"1,406 km",883 km,"1,015 km"
4,Moyobamba,673 km,"1,291 km","2,173 km","1,335 km",0 km,"1,871 km","1,487 km","1,015 km",-,...,"2,095 km",425 km,"2,382 km","1,104 km",850 km,-,"1,916 km",716 km,"1,955 km",889 km
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,Putumayo,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,0 km,-,-,-,-
192,Urubamba,"1,810 km","1,127 km",267 km,749 km,"1,921 km",743 km,750 km,"1,401 km",-,...,344 km,"1,496 km",427 km,"1,103 km","1,333 km",-,0 km,"2,133 km",86.3 km,"1,742 km"
193,Sullana,324 km,"1,217 km","2,099 km","1,518 km",716 km,"1,797 km","1,412 km",914 km,-,...,"2,021 km",996 km,"2,308 km","1,030 km",957 km,-,"2,124 km",0 km,"2,163 km",541 km
194,Quispicanchi,"1,849 km","1,166 km",180 km,788 km,"1,960 km",837 km,789 km,"1,440 km",-,...,258 km,"1,535 km",341 km,"1,142 km","1,372 km",-,86.1 km,"2,172 km",0 km,"1,781 km"


In [152]:
# prov_distances.to_excel('prov_distances.xlsx', index=False)
# prov_times.to_excel('prov_times.xlsx', index=False)

### Aplicando la función a las Matrices de Distritos

In [53]:
# kappa = gmaps.distance_matrix(origins=origen_destino_dist[0], destinations=origen_destino_dist[1], mode='driving')
kappa
origen_destino_dist[0]
# origen_destino_dist[1]

'Coya+Calca+Cusco+Peru'

In [136]:
primera_fila = 1
primera_col = 1
ult_fila_mas_uno = 2
ult_col = len(distritos)+1


# matrix_blocks(primera_fila, primera_col, ult_fila_mas_uno, ult_col, dist_distances, dist_times, origen_destino_dist)

1


In [140]:
dist_distances.head()

,Origin Province\Destination Province,Pariahuanca,Sondorillo,Tocache,Tintay Puncu,Curimana,Chazuta,Candarave,Marcara,Coyllurqui,...,Cochorco,Acos Vinchos,Colcamar,Pillpinto,Oropesa,Yauyos,La Banda de Shilcayo,Chirinos,Putumayo,Velille
0,Pariahuanca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sondorillo,"1,382 km",0 km,918 km,-,"1,289 km",657 km,"2,324 km",778 km,"2,089 km",...,790 km,"1,641 km",354 km,"2,248 km","2,173 km","1,336 km",609 km,146 km,-,"2,129 km"
2,Tocache,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Tintay Puncu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Curimana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
